# Building a regression model for predicting house sale prices

In [1]:
import pickle
import pathlib

import numpy as np
import pandas as pd

In [2]:
DATA_DIR = pathlib.Path.cwd().parent / 'data'
print(DATA_DIR)

c:\Users\jalfr\Desktop\Machine Learning\Bangu1\ames-Bangu1\data


In [3]:
clean_data_path = DATA_DIR / 'processed' / 'ames_clean.pkl'

In [4]:
with open(clean_data_path, 'rb') as file:
    data = pickle.load(file)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2877 entries, 0 to 2929
Data columns (total 70 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   MS.SubClass      2877 non-null   category
 1   MS.Zoning        2877 non-null   category
 2   Lot.Frontage     2877 non-null   float64 
 3   Lot.Area         2877 non-null   float64 
 4   Lot.Shape        2877 non-null   category
 5   Land.Contour     2877 non-null   category
 6   Lot.Config       2877 non-null   category
 7   Land.Slope       2877 non-null   category
 8   Neighborhood     2877 non-null   category
 9   Bldg.Type        2877 non-null   category
 10  House.Style      2877 non-null   category
 11  Overall.Qual     2877 non-null   category
 12  Overall.Cond     2877 non-null   category
 13  Roof.Style       2877 non-null   category
 14  Mas.Vnr.Type     2877 non-null   category
 15  Mas.Vnr.Area     2877 non-null   float64 
 16  Exter.Qual       2877 non-null   category
 17  

In [6]:
model_data = data.copy()

## Preparing the data for the model

Now that we have the data all cleaned, and all the missing values accounted for, lets focus on transforming the data for the model.

Lets remember what a model is. 

- A predictive model is a **set** of functions that receive data as input and produce a prediction, that is, an estimate of the target value as output.
- **Train** a model is to search the set of candidate functions for one that adequately represents the **training dataset**.
- The adequacy of a candidate function to the training data is usually determined by a **loss function** that measures how well the predictions of the function match the real values of the target within the training dataset. It is common to define a *loss function per data item* (e.g. absolute error, quadratic error, etc) and to construct the *loss function over the dataset* as the *average prediction loss*.

Many models are **parametric models**. In this case, each function of the set of functions that makes the model is constructed from a vector of **parameters** that define the function, forming a **parametric function**. For instance: the linear model constructs prediction values out of a linear combination of the input features, plus a constant. The weights of the linear combination plus the constant are the parameters of the model. The set of functions that can be represented by this model is given by all possible values of the vector of parameters that define the function.

Some models are called **non-parametric models**. These models usually do not have a parametric form (like the linear model). But the terminology is a bit misleading, though: usually these models *do* have parameters, and potentially an open-ended set of them! For instance, consider the "decision tree" model, which is one of the most prominent models of this category. The decision tree may not have a formula for the predicted value, but it does have parameters, many of them: each decision in the tree involves a choice of feature and a threshold level, and those choices must be stored as parameters of the model for use in future predictions.

Each model has specific requirements for the format of the input data. Most of the time, the minimum requirement is that:

- All columns are numeric;
- There are no missing values.

Some models have extra requirements. For example: the support-vector-machines model requires that the input features have comparable standard deviations - having features that have large discrepancies between features in terms of their order of magnitude (such as a feature in the fractions of unit range and another in the tens of thousands) will result in poor prediction quality.

And some models may not have any special requirement at all. We will study each of those in detail in this course.

Lets start our study with a simple model: the *multivariate linear regression* model. This is a model that presents the minimum requirements listed above. So we need to do a bit of processing on the original features:

- *Numerical features* stay as given;
- *Categorical features* have to be transformed into numerical features. In order to do so we need to **encode** these features, that is: to transform them into new features that convey the same information, but in a numerical form, and in a way that "makes sense" - we'll see it below.
- *Ordinal features* can be transformed into numerical features in the same way as the caegorical features, or could be assigned increasing numbers in conformity with the ordered nature of the categories of the feature.

## Encoding categorical variables

Lets identify all categorical variables - both nominal (that is, categoricals without category order) and ordinal.

In [7]:
categorical_columns = []
ordinal_columns = []
for col in model_data.select_dtypes('category').columns:
    if model_data[col].cat.ordered:
        ordinal_columns.append(col)
    else:
        categorical_columns.append(col)

In [8]:
ordinal_columns

['Lot.Shape',
 'Land.Slope',
 'Overall.Qual',
 'Overall.Cond',
 'Exter.Qual',
 'Exter.Cond',
 'Heating.QC',
 'Electrical',
 'Kitchen.Qual',
 'Functional',
 'Paved.Drive',
 'Fence']

In [9]:
categorical_columns

['MS.SubClass',
 'MS.Zoning',
 'Land.Contour',
 'Lot.Config',
 'Neighborhood',
 'Bldg.Type',
 'House.Style',
 'Roof.Style',
 'Mas.Vnr.Type',
 'Foundation',
 'Bsmt.Qual',
 'Bsmt.Cond',
 'Bsmt.Exposure',
 'BsmtFin.Type.1',
 'BsmtFin.Type.2',
 'Central.Air',
 'Garage.Type',
 'Garage.Finish',
 'Sale.Type',
 'Sale.Condition',
 'Condition',
 'Exterior']

### Encoding ordinal variables 

Ordinal variables can be transformed into integer numbers in a straightforward manner: the lowest category is assigned the value "zero", the next category is given the value "one", etc. The `Pandas` library has a function for this task: `factorize()`:

In [10]:
for col in ordinal_columns:
    codes, _ = pd.factorize(data[col], sort=True)
    model_data[col] = codes

Lets confirm that the variables are no longer ordinal, but now are integers:

In [11]:
model_data[ordinal_columns].info()

<class 'pandas.core.frame.DataFrame'>
Index: 2877 entries, 0 to 2929
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   Lot.Shape     2877 non-null   int64
 1   Land.Slope    2877 non-null   int64
 2   Overall.Qual  2877 non-null   int64
 3   Overall.Cond  2877 non-null   int64
 4   Exter.Qual    2877 non-null   int64
 5   Exter.Cond    2877 non-null   int64
 6   Heating.QC    2877 non-null   int64
 7   Electrical    2877 non-null   int64
 8   Kitchen.Qual  2877 non-null   int64
 9   Functional    2877 non-null   int64
 10  Paved.Drive   2877 non-null   int64
 11  Fence         2877 non-null   int64
dtypes: int64(12)
memory usage: 292.2 KB


Compare the original values with the encoded values:

In [12]:
data['Lot.Shape'].value_counts()

Lot.Shape
Reg    1825
IR1     960
IR2      76
IR3      16
Name: count, dtype: int64

In [13]:
model_data['Lot.Shape'].value_counts()

Lot.Shape
0    1825
1     960
2      76
3      16
Name: count, dtype: int64

### Encoding nominal variables

With nominal variables there is no notion of order among categories. Therefore, it would be a conceptual mistake to encode them in the same manner as the ordinal variables. For instance, consider the `Exterior` variable:

In [14]:
model_data['Exterior'].value_counts()

Exterior
VinylSd    1024
HdBoard     439
MetalSd     432
Wd Sdng     401
Plywood     218
CemntBd     126
BrkFace      86
WdShing      55
Stucco       42
AsbShng      41
Other        13
Name: count, dtype: int64

We cannot assign an order here, lest we end up with equations like `HdBoard` + `Plywood` = `CemntBd`, which are nonsense. 

The strategy here to encode `Exterior` is to create several new numerical variables to represent the membership of a given data item to one of the `Exterior` categories. These are called **dummy variables**. Each of these new variables contain only the values "zero" or "one" (i.e. they are binary variables), where $1$ denotes that the data item belongs to the category represented by the variable. Evidently, for a given data item, only one dummy variable has a value of $1$, all remaining are $0$.

There are two types of dummy variable encoding:

- "One-hot" encoding: in this case we create one dummy variable per category. Let's look at the `Exterior` feature as an example. The `Pandas` function `get_dummies()` can do the encoding for us:

In [15]:
original_data = model_data['Exterior']
encoded_data = pd.get_dummies(original_data)

aux_dataframe = encoded_data
aux_dataframe['Exterior'] = original_data.copy()

aux_dataframe.head().transpose()

,0,1,2,3,4
AsbShng,False,False,False,False,False
BrkFace,True,False,False,True,False
CemntBd,False,False,False,False,False
HdBoard,False,False,False,False,False
MetalSd,False,False,False,False,False
Plywood,False,False,False,False,False
Stucco,False,False,False,False,False
VinylSd,False,True,False,False,True
Wd Sdng,False,False,True,False,False
WdShing,False,False,False,False,False


Observe that for each value of `Exterior`, only the corresponding dummy is flagged.

One-hot encoding is a popular technique in Machine Learning. Statisticians, however, prefer a slightly different way of dummy encoding which is:

- Choose a category to *not encode* (this is called the *base category*)
- Generate dummies for the remaining categories. That is:
    - If the data item belongs to the base category, no dummy receives a value of $1$;
    - Otherwise, set the corresponding dummy to $1$.

The same `get_dummies()` function of `Pandas` can do this automatically with the `drop_first` argument:

In [16]:
original_data = model_data['Exterior']
encoded_data = pd.get_dummies(original_data, drop_first=True)

aux_dataframe = encoded_data
aux_dataframe['Exterior'] = original_data.copy()

aux_dataframe.head().transpose()

,0,1,2,3,4
BrkFace,True,False,False,True,False
CemntBd,False,False,False,False,False
HdBoard,False,False,False,False,False
MetalSd,False,False,False,False,False
Plywood,False,False,False,False,False
Stucco,False,False,False,False,False
VinylSd,False,True,False,False,True
Wd Sdng,False,False,True,False,False
WdShing,False,False,False,False,False
Other,False,False,False,False,False


Notice that we are now missing the dummy variable for the `AsbShng` category.

Why to encode things this way? If we don't drop one of the dummies, then it will always be the case that the sum of the values of the dummies is $1$ (since each data item must belong to one of the categories). The linear model, particularly very popular with the statisticians, implies the existence of a fictitious feature containing, for all data items, the value $1$. Hence we end up having a set of variables where a linear combination of them (in this case, the sum of the dummies) matches the value at another variable. This has numerical computing implications for the linear model, that we will discuss in class.

Since we want to use the linear model in this notebook, lets encode all categoricals with the `drop_first` alternative.

In [17]:
model_data = pd.get_dummies(model_data, drop_first=True)

Now our dataset has a lot more variables!

In [18]:
model_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2877 entries, 0 to 2929
Columns: 165 entries, Lot.Frontage to Exterior_Other
dtypes: bool(119), float64(34), int64(12)
memory usage: 1.4 MB


In [19]:
for cat in categorical_columns:
    dummies = []
    for col in model_data.columns:
        if col.startswith(cat + "_"):
            dummies.append(f'"{col}"')
    dummies_str = ', '.join(dummies)
    print(f'From column "{cat}" we made {dummies_str}\n')

From column "MS.SubClass" we made "MS.SubClass_30", "MS.SubClass_50", "MS.SubClass_60", "MS.SubClass_70", "MS.SubClass_80", "MS.SubClass_85", "MS.SubClass_90", "MS.SubClass_120", "MS.SubClass_160", "MS.SubClass_190", "MS.SubClass_Other"

From column "MS.Zoning" we made "MS.Zoning_RH", "MS.Zoning_RL", "MS.Zoning_RM"

From column "Land.Contour" we made "Land.Contour_HLS", "Land.Contour_Low", "Land.Contour_Lvl"

From column "Lot.Config" we made "Lot.Config_CulDSac", "Lot.Config_FR2", "Lot.Config_FR3", "Lot.Config_Inside"

From column "Neighborhood" we made "Neighborhood_BrDale", "Neighborhood_BrkSide", "Neighborhood_ClearCr", "Neighborhood_CollgCr", "Neighborhood_Crawfor", "Neighborhood_Edwards", "Neighborhood_Gilbert", "Neighborhood_IDOTRR", "Neighborhood_MeadowV", "Neighborhood_Mitchel", "Neighborhood_NAmes", "Neighborhood_NPkVill", "Neighborhood_NWAmes", "Neighborhood_NoRidge", "Neighborhood_NridgHt", "Neighborhood_OldTown", "Neighborhood_SWISU", "Neighborhood_Sawyer", "Neighborhood_Sa

## Train-test splitting

The data will now be organized as follows:

- The features form a matrix $X$ of size $m \times n$, where $m$ is the number of data items, and $n$ is the number of features.
- The target forms a column-matrix $y$ of length $m$.

In [20]:
X = model_data.drop(columns=['SalePrice']).copy()
y = model_data['SalePrice'].copy()

In [21]:
X.values, y.values

(array([[141.0, 31770.0, 1, ..., False, False, False],
        [80.0, 11622.0, 0, ..., False, False, False],
        [81.0, 14267.0, 1, ..., True, False, False],
        ...,
        [62.0, 10441.0, 0, ..., False, False, False],
        [77.0, 10010.0, 0, ..., False, False, False],
        [74.0, 9627.0, 0, ..., False, False, False]], dtype=object),
 array([5.33243846, 5.0211893 , 5.23552845, ..., 5.12057393, 5.23044892,
        5.27415785]))

This is the typical set-up of a machine learning project. Now we want to train our model *and* verify that the model provides good predictions for *unseen* data items. Why the emphasis on "unseen"? Because there is no use for a model that only gives predictions for the items in the data used to train it - we want our models to *generalize*.

The way to assess the model's performance for unseen values is to split the dataset into two subsets: the **training** and **test** datasets.

We have been using a lot of `Pandas` to manipulate our data so far. From now on we will switch to another very popular library for machine learning in Python: `Scikit-Learn`.

The function `train_test_split()` will take as arguments the dataset to be split, the specification of the fraction of the dataset to be reserved for testing, and a random seed value - so that the split will always be the same whenever we run our notebook. This is a customary measure to ensure reproducibility of the notebook.

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
RANDOM_SEED = 42  # Any number here, really.

In [24]:
Xtrain, Xtest, ytrain, ytest = train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=RANDOM_SEED,
)


In [25]:
X.shape, Xtrain.shape, Xtest.shape

((2877, 164), (2157, 164), (720, 164))

In [26]:
y.shape, ytrain.shape, ytest.shape

((2877,), (2157,), (720,))

## Fitting a linear model

Lets start with fitting a linear model for regression. The linear model is one of the oldest and most used models for regression, due to its simplicity and strong statistical roots. A proper statistical approach to the understanding of the linear model consists of:

- Understanding the statistical premises of the linear model;
- Analyzing the features to verify that the preliminary conditions of this modeling strategy are satisfied;
- Fitting the model;
- Analyzing the residuals to confirm that the post-fit conditions are satisfied.

Lets discuss these topics in more detail:

### The statistical approach to the linear model

In machine learning we are more interested in the predictive capability of a model, rather than its inductive use to analyze the relation between features and target (or independent and dependent variables, in the statistical terminology). But even to a machine learning practitioner, understanding the statistical basis of the linear model may lead to better predictive performance. For instance:

- Having a symmetrical residual is usually associated with better mean-squared-error (MSE) than having a long-tailed assymmetric residual;
- Non-significant parameters (in a hypothesis-testing sense) may indicate superfluous variables in the model, which could be associated with reduced performance in the test dataset (i.e. poor generalization).

So what is the linear model in statistics? A statistical model is a way to describe probabilistically the relation between features and targets, usually in a parametric way.

Mathematically, we are searching for a *conditional probability* density model of the form $f(Y = y | \mathbf{x}, \theta)$ where $\mathbf{x}$ is the feature vector, $Y$ is the random variable associated with the target, and $\theta$ is the vector of parameters. In plain language, we would like to describe the probability distribution of the target variable when the value of the feature vector is known and the parameters of the model are given.

In the linear model, we postulate that the data $y$ is generated from the feature vector $\mathbf{x}$ plus a random Gaussian noise of fixed standard deviation, as shown in the equation below:

$$
y = \theta_0 + \theta_1 x_1 + \theta_2 x_2 + \cdots + \theta_n x_n + \varepsilon
$$

where $\varepsilon \sim N(0, \sigma)$

The addition of the noise term causes the $y$ value to become a random variable itself, thus making the probabilistic model mentioned above. For a given value of $\mathbf{x}$, the value of $y$ is obtained by adding the constant value $\theta_0 + \theta_1 x_1 + \theta_2 x_2 + \cdots + \theta_n x_n$ to a normal random variable $\varepsilon$. Remember that, for normal random variables, adding a constant keeps the variable normal, with a shifted mean-value parameter. Therefore:

$$
Y \sim N(\mu = (\theta_0 + \theta_1 x_1 + \theta_2 x_2 + \cdots + \theta_n x_n), \sigma)
$$

Lets write

$$
\hat{y} = \theta_0 + \theta_1 x_1 + \theta_2 x_2 + \cdots + \theta_n x_n
$$

for simplicity, then the model above is rewritten as:

$$
Y \sim N(\mu = \hat{y}, \sigma)
$$

When we have a dataset $D = \{(\mathbf{x}_1, y_1), (\mathbf{x}_2, y_2), \cdots, (\mathbf{x}_m, y_m)\}$ of several $(\mathbf{x}, y)$ pairs, what is their joint conditional probability density function $f(y_1, y_2, \cdots, y_m | x_1, x_2, \cdots, x_n, \theta)$?

We will make another supposition of the linear model: that the $(\mathbf{x}, y)$ examples were obtained independently, and that the value of one does not impact the probability of the other. Therefore:

$$
f(y_1, y_2, \cdots, y_m | x_1, x_2, \cdots, x_n, \theta) = f(y_1| x_1, \theta) f(y_2| x_2, \theta) \cdots f(y_m| x_m, \theta) 
$$

Remember that the normal probability density function is as follows:

$$
Y \sim N(\mu, \sigma) \Rightarrow f(y) = \frac{1}{\sigma \sqrt{2 \pi}} \exp \left(-\frac{1}{2}\frac{(y - \mu)^2}{\sigma^2} \right)
$$

Thus:

$$
Y \sim N(\mu = \hat{y}, \sigma) \Rightarrow f(y) = \frac{1}{\sigma \sqrt{2 \pi}} \exp \left(-\frac{1}{2}\frac{(y - \hat{y})^2}{\sigma^2} \right)
$$

And the joint conditional probability density function of the entire dataset becomes:


$$
f(y_1, y_2, \cdots, y_m | x_1, x_2, \cdots, x_n, \theta) = \prod_{i=1}^{m} \left(\frac{1}{\sigma \sqrt{2 \pi}} \exp \left(-\frac{1}{2}\frac{(y_i - \hat{y_i})^2}{\sigma^2} \right) \right)
$$

Expanding the product we have:

$$
\begin{align*}
f(y_1, y_2, \cdots, y_m | x_1, x_2, \cdots, x_n, \theta) & = & \prod_{i=1}^{m} \left(\frac{1}{\sigma \sqrt{2 \pi}} \exp \left(-\frac{1}{2}\frac{(y_i - \hat{y_i})^2}{\sigma^2} \right) \right) \\
& = & \prod_{i=1}^{m} \left(\frac{1}{\sigma \sqrt{2 \pi}} \right) \prod_{i=1}^{m} \left( \exp \left(-\frac{1}{2}\frac{(y_i - \hat{y_i})^2}{\sigma^2} \right) \right) \\
& = & \left(\frac{1}{\sigma \sqrt{2 \pi}} \right)^{m} \exp \left(\sum_{i=1}^{m} \left(-\frac{1}{2}\frac{(y_i - \hat{y_i})^2}{\sigma^2} \right) \right) \\
& = & \left(\frac{1}{\sigma \sqrt{2 \pi}} \right)^{m} \exp \left(- \frac{1}{2 \sigma} \sum_{i=1}^{m} (y_i - \hat{y_i})^2 \right)
\end{align*}
$$

What are the "best" value for the parameters of the linear model? We can search for the parameters that maximize the joint conditional probability density function of the dataset. This function is called the *likelihood* of the parameters, and therefore our solution here is called a *"maximum likelihood estimate"* of the parameters.

So we are looking for a value $\theta^{\star}$ of $\theta$ to maximize $f(y_1, y_2, \cdots, y_m | x_1, x_2, \cdots, x_n, \theta)$, that is:

$$
\begin{align*}
\theta^{\star} & = & \argmax_{\theta} \left\{ f(y_1, y_2, \cdots, y_m | x_1, x_2, \cdots, x_n, \theta) \right\}\\
& = & \argmax_{\theta} \left\{ \left(\frac{1}{\sigma \sqrt{2 \pi}} \right)^{m} \exp \left(- \frac{1}{2 \sigma} \sum_{i=1}^{m} (y_i - \hat{y_i})^2 \right) \right\} \\
& = & \argmax_{\theta} \left\{ \exp \left(- \frac{1}{2 \sigma} \sum_{i=1}^{m} (y_i - \hat{y_i})^2 \right) \right\} \\
& = & \argmax_{\theta} \left\{ - \frac{1}{2 \sigma} \sum_{i=1}^{m} (y_i - \hat{y_i})^2 \right\} \\
& = & \argmin_{\theta} \left\{ \frac{1}{2 \sigma} \sum_{i=1}^{m} (y_i - \hat{y_i})^2 \right\} \\
& = & \argmin_{\theta} \left\{ \sum_{i=1}^{m} (y_i - \hat{y_i})^2 \right\} \\
& = & \argmin_{\theta} \left\{ \frac{1}{m} \sum_{i=1}^{m} (y_i - \hat{y_i})^2 \right\} \\
\end{align*}
$$

Hey, look who we found! Our old friend MSE (mean-squared-error)!

So, in the end, here are the lessons:

- The statistical formulation of the linear model leads to the same error formulation of machine learning, which only cares for the prediction quality.
- The statistical linear model has several assumptions:
    - The samples are independent;
    - The target is *truly* generated from the linear predictive formula plus a normally-distributed error;
    - The error has zero mean and constant standard deviation (the *homoscedasticity* hypothesis);
    - There is no error in the feature measurement. That is, $\mathbf{x}_{i}$ are constants, not random variables. All the error is in the target;
- If the assumptions of the linear model are satisfied, you can analyze the parameters with greater sophistication (the machine learning formulation does not bring this finesse). For instance, you can run hypothesis tests on the values of the parameters to determine whether they refute the null hypothesis $\theta_i = 0$ with a given statistical significance level. Which, in plain language, means that you don't really trust that the associated feature impacts the target, or if it is just an accident.

In [27]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

In [28]:
# Teste com todos os modelos

linreg = LinearRegression()
ridge = Ridge()
lasso = Lasso()
elasticNet = ElasticNet()

linreg.fit(Xtrain, ytrain)
ridge.fit(Xtrain, ytrain)
lasso.fit(Xtrain, ytrain)
elasticNet.fit(Xtrain, ytrain)

ElasticNet()

In [29]:
pred_linreg = linreg.predict(Xtest)
pred_ridge = ridge.predict(Xtest)
pred_lasso = lasso.predict(Xtest)
pred_elasticNet = elasticNet.predict(Xtest)

In [30]:
from sklearn.metrics import mean_squared_error

RMSE_lin = np.sqrt(mean_squared_error(ytest, pred_linreg))
RMSE_ridge = np.sqrt(mean_squared_error(ytest, pred_ridge))
RMSE_lasso = np.sqrt(mean_squared_error(ytest, pred_lasso))
RMSE_elasticNet = np.sqrt(mean_squared_error(ytest, pred_elasticNet))


In [31]:
error_percent = 100 * (10**RMSE_lin - 1)
print(f'Average error for LinearRegression is {error_percent:.2f}%')

error_percent = 100 * (10**RMSE_ridge - 1)
print(f'Average error for Ridge is {error_percent:.2f}%')

error_percent = 100 * (10**RMSE_lasso - 1)
print(f'Average error for Lasso is {error_percent:.2f}%')

error_percent = 100 * (10**RMSE_elasticNet - 1)
print(f'Average error for ElasticNet is {error_percent:.2f}%')

Average error for LinearRegression is 38759319810481081068002167750656.00%
Average error for Ridge is 15.10%
Average error for Lasso is 27.56%
Average error for ElasticNet is 25.21%


In [32]:
from sklearn.discriminant_analysis import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

pipeLin = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('cls', LinearRegression())
])

pipeLin.fit(Xtrain,ytrain)
pred_pipeLin = pipeLin.predict(Xtest)


from sklearn.metrics import mean_squared_error

RMSE_pipeLin = np.sqrt(mean_squared_error(ytest, pred_pipeLin))

error_percent = 100 * (10**RMSE_pipeLin - 1)
print(f'Average error for LinearRegression Pipeline is {error_percent:.2f}%')

Average error for LinearRegression Pipeline is 22.36%


In [33]:
from sklearn.discriminant_analysis import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

pipeRidge = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('cls', Ridge())
])

pipeRidge.fit(Xtrain,ytrain)
pred_pipeRidge = pipeRidge.predict(Xtest)


from sklearn.metrics import mean_squared_error

RMSE_pipeRidge = np.sqrt(mean_squared_error(ytest, pred_pipeRidge))

error_percent = 100 * (10**RMSE_pipeRidge - 1)
print(f'Average error for Ridge Pipeline is {error_percent:.2f}%')

Average error for Ridge Pipeline is 21.62%


**STDScaler** - 

Lot.Area

X1st.Flr.SF

Bsmt.Unf.SF

Total.Bsmt.SF

Lot.Frontage

-------------------------------------------------------------------------------------

**Year.Remod** - Fazer coluna de se teve remodelling ou não e retirar o House Age

-------------------------------------------------------------------------------------

**Droppar** - 

Low.Qual.Fin

X2nd.Flr.SF

3ssn.Porch

BsmtFin.SF.2

Bsmt.Half.Bath

Mas.Vnr.Area

-------------------------------------------------------------------------------------

**Enclosed.Porch** - Tem ou não Tem

-------------------------------------------------------------------------------------

**Screen.Porc** - Tem ou não tem

In [34]:
X = model_data.drop(columns=['SalePrice', 'Low.Qual.Fin.SF', 'X2nd.Flr.SF', 'X3Ssn.Porch', 'BsmtFin.SF.2', 'Bsmt.Half.Bath', 'Mas.Vnr.Area']).copy()
y = model_data['SalePrice'].copy()

In [35]:
list(X.columns)

['Lot.Frontage',
 'Lot.Area',
 'Lot.Shape',
 'Land.Slope',
 'Overall.Qual',
 'Overall.Cond',
 'Exter.Qual',
 'Exter.Cond',
 'BsmtFin.SF.1',
 'Bsmt.Unf.SF',
 'Total.Bsmt.SF',
 'Heating.QC',
 'Electrical',
 'X1st.Flr.SF',
 'Gr.Liv.Area',
 'Bsmt.Full.Bath',
 'Full.Bath',
 'Half.Bath',
 'Bedroom.AbvGr',
 'Kitchen.AbvGr',
 'Kitchen.Qual',
 'TotRms.AbvGrd',
 'Functional',
 'Fireplaces',
 'Garage.Cars',
 'Garage.Area',
 'Paved.Drive',
 'Wood.Deck.SF',
 'Open.Porch.SF',
 'Enclosed.Porch',
 'Screen.Porch',
 'Pool.Area',
 'Fence',
 'Misc.Val',
 'Mo.Sold',
 'Yr.Sold',
 'HasShed',
 'HasAlley',
 'Garage.Age',
 'Remod.Age',
 'House.Age',
 'MS.SubClass_30',
 'MS.SubClass_50',
 'MS.SubClass_60',
 'MS.SubClass_70',
 'MS.SubClass_80',
 'MS.SubClass_85',
 'MS.SubClass_90',
 'MS.SubClass_120',
 'MS.SubClass_160',
 'MS.SubClass_190',
 'MS.SubClass_Other',
 'MS.Zoning_RH',
 'MS.Zoning_RL',
 'MS.Zoning_RM',
 'Land.Contour_HLS',
 'Land.Contour_Low',
 'Land.Contour_Lvl',
 'Lot.Config_CulDSac',
 'Lot.Config_FR2

In [36]:
X['Enclosed.Porch']

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
2925    0.0
2926    0.0
2927    0.0
2928    0.0
2929    0.0
Name: Enclosed.Porch, Length: 2877, dtype: float64

In [37]:
X['Screen.Porch']

0         0.0
1       120.0
2         0.0
3         0.0
4         0.0
        ...  
2925      0.0
2926      0.0
2927      0.0
2928      0.0
2929      0.0
Name: Screen.Porch, Length: 2877, dtype: float64

In [38]:
M = X.to_numpy()

M[:, 29] == np.zeros(M[:, 29].shape)

array([ True,  True,  True, ...,  True,  True,  True])

Criar coluna se a casa foi reformada ou não 


Drop a coluna de idade de reforma 


Verificar as colunas para aplicar o stdsc 


Garage age = garageage/houseage


Garage cars drop


Bsmt Unf SF dividido por BsmtFin SF 1

In [39]:
"House.Age", "Remod.Age", "Enclosed.Porch", "Screen.Porch", "Garage.Cars", "Garage.Age", "Bsmt.Unf.SF", "BsmtFin.SF.1"

('House.Age',
 'Remod.Age',
 'Enclosed.Porch',
 'Screen.Porch',
 'Garage.Cars',
 'Garage.Age',
 'Bsmt.Unf.SF',
 'BsmtFin.SF.1')

In [40]:
list(X.columns).index("House.Age"), list(X.columns).index("Remod.Age"), list(X.columns).index("Enclosed.Porch"), list(X.columns).index("Screen.Porch"), list(X.columns).index("Garage.Cars"), list(X.columns).index("Garage.Age"), list(X.columns).index("Bsmt.Unf.SF"), list(X.columns).index("BsmtFin.SF.1")

(40, 39, 29, 30, 24, 38, 9, 8)

In [83]:
from sklearn.model_selection import train_test_split
RANDOM_SEED = 42  # Any number here, really.

Xtrain, Xtest, ytrain, ytest = train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=RANDOM_SEED,
)

In [71]:
from sklearn.metrics import mean_squared_error

ridge = Ridge()

ridge.fit(Xtrain, ytrain)

pred_ridge = ridge.predict(Xtest)

RMSE_ridge = np.sqrt(mean_squared_error(ytest, pred_ridge))

error_percent = 100 * (10**RMSE_ridge - 1)
print(f'Average error for Ridge is {error_percent:.2f}%')

Average error for Ridge is 15.03%


In [72]:
from sklearn.discriminant_analysis import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

pipeRidge = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('cls', Ridge())
])

pipeRidge.fit(Xtrain,ytrain)
pred_pipeRidge = pipeRidge.predict(Xtest)


from sklearn.metrics import mean_squared_error

RMSE_pipeRidge = np.sqrt(mean_squared_error(ytest, pred_pipeRidge))

error_percent = 100 * (10**RMSE_pipeRidge - 1)
print(f'Average error for Ridge Pipeline is {error_percent:.2f}%')

Average error for Ridge Pipeline is 22.55%


In [84]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    # Column index.
    house_age, remodel, enclosed, screen, garage_cars, garage_age, bsmt_unf, bsmt_fin = "House.Age", "Remod.Age", "Enclosed.Porch", "Screen.Porch", "Garage.Cars", "Garage.Age", "Bsmt.Unf.SF", "BsmtFin.SF.1"
    
    def __init__(self):  # No *args or **kwargs.
        pass
            
    def fit(self, X, y=None):
        return self  # Nothing else to do.

    def transform(self, X, y=None):

        df = X.copy()

        df['has_remodel'] = \
            df[CombinedAttributesAdder.remodel] == df[CombinedAttributesAdder.house_age]
        
        df['has_enclosed_porch'] = \
            df[CombinedAttributesAdder.enclosed].to_numpy() == np.zeros(df[CombinedAttributesAdder.enclosed].shape)

        df['has_screen_porch'] = \
            df[CombinedAttributesAdder.screen].to_numpy() == np.zeros(df[CombinedAttributesAdder.screen].shape)
        
        df['garage_by_house_age'] = \
            df[CombinedAttributesAdder.garage_age] / df[CombinedAttributesAdder.house_age]
        
        df['bsmt_unf_by_fin'] = \
            df[CombinedAttributesAdder.bsmt_unf] / df[CombinedAttributesAdder.bsmt_fin]
        
        df = df.drop(columns = ["Remod.Age", "Enclosed.Porch", "Screen.Porch", "Garage.Cars", "Bsmt.Unf.SF", "BsmtFin.SF.1"])

        return df.astype(float)
    


columns_std = ['X1st.Flr.SF','Total.Bsmt.SF','Lot.Frontage']


all_columns = list(X.columns)

adder = Pipeline([
        ('attribs_adder', CombinedAttributesAdder()),
    ])

finishing = Pipeline([
        ('poly', PolynomialFeatures(degree=2, include_bias=False))
    ])

full_pipeline = ColumnTransformer([
        ("scaler", StandardScaler(), columns_std),
        ("transform", adder, X.columns)
    ], remainder="passthrough")

Xtrain_transformed = full_pipeline.fit_transform(Xtrain)

Xtrain_transformed.shape

# pipeRidge = Pipeline([
#     # ('scaler', StandardScaler()),
#     ('poly', PolynomialFeatures(degree=2, include_bias=False)),
#     ('cls', Ridge())
# ])

# pipeRidge.fit(Xtrain_transformed,ytrain)

# Xtest_transformed = full_pipeline.transform(Xtest)
# pred_pipeRidge = pipeRidge.predict(Xtest_transformed)


# from sklearn.metrics import mean_squared_error

# RMSE_pipeRidge = np.sqrt(mean_squared_error(ytest, pred_pipeRidge))

# error_percent = 100 * (10**RMSE_pipeRidge - 1)
# print(f'Average error for Ridge Pipeline is {error_percent:.2f}%')

(2157, 160)

In [45]:
Xtrain_transformed

array([[-0.13746726,  0.45301543,  1.75314085, ...,  1.        ,
         0.        ,         inf],
       [-0.2212866 , -0.54188073, -1.26865837, ...,  1.        ,
         1.        ,  0.        ],
       [-0.37778928,  0.19584906, -0.25686492, ...,  1.        ,
         1.        ,  0.96129032],
       ...,
       [ 0.33766863, -0.64578633, -0.87254237, ...,  0.        ,
         1.        ,  0.23744912],
       [-0.49669299, -0.71852026,  0.02607508, ...,  1.        ,
         0.27184466,         inf],
       [-0.09256405, -0.26912852, -0.548859  , ...,  1.        ,
         0.91666667,  0.43031123]])

In [90]:
features_test_corr = model_data[["Lot.Frontage","Mas.Vnr.Area","BsmtFin.SF.1","Total.Bsmt.SF","X1st.Flr.SF","Gr.Liv.Area","Full.Bath","TotRms.AbvGrd","Fireplaces","Garage.Cars","Garage.Area","Garage.Age","Remod.Age","House.Age"]]
features_test_corr

,Lot.Frontage,Mas.Vnr.Area,BsmtFin.SF.1,Total.Bsmt.SF,X1st.Flr.SF,Gr.Liv.Area,Full.Bath,TotRms.AbvGrd,Fireplaces,Garage.Cars,Garage.Area,Garage.Age,Remod.Age,House.Age
0,141.0,112.0,639.0,1080.0,1656.0,1656.0,1.0,7.0,2.0,2.0,528.0,50.0,50.0,50.0
1,80.0,0.0,468.0,882.0,896.0,896.0,1.0,5.0,0.0,1.0,730.0,49.0,49.0,49.0
2,81.0,108.0,923.0,1329.0,1329.0,1329.0,1.0,6.0,0.0,1.0,312.0,52.0,52.0,52.0
3,93.0,0.0,1065.0,2110.0,2110.0,2110.0,2.0,8.0,2.0,2.0,522.0,42.0,42.0,42.0
4,74.0,0.0,791.0,928.0,928.0,1629.0,2.0,6.0,1.0,2.0,482.0,13.0,12.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,37.0,0.0,819.0,1003.0,1003.0,1003.0,1.0,6.0,0.0,2.0,588.0,22.0,22.0,22.0
2926,68.0,0.0,301.0,864.0,902.0,902.0,1.0,5.0,0.0,2.0,484.0,23.0,23.0,23.0
2927,62.0,0.0,337.0,912.0,970.0,970.0,1.0,6.0,0.0,0.0,0.0,28.0,14.0,14.0
2928,77.0,0.0,1071.0,1389.0,1389.0,1389.0,1.0,6.0,1.0,2.0,418.0,31.0,31.0,32.0


In [91]:
from sklearn.model_selection import train_test_split
RANDOM_SEED = 42  # Any number here, really.

Xtrain, Xtest, ytrain, ytest = train_test_split(
    features_test_corr,
    y,
    test_size=0.25,
    random_state=RANDOM_SEED,
)

In [92]:
from sklearn.metrics import mean_squared_error

ridge = Ridge()

ridge.fit(Xtrain, ytrain)

pred_ridge = ridge.predict(Xtest)

RMSE_ridge = np.sqrt(mean_squared_error(ytest, pred_ridge))

error_percent = 100 * (10**RMSE_ridge - 1)
print(f'Average error for Ridge is {error_percent:.2f}%')

Average error for Ridge is 22.90%


In [93]:
from sklearn.discriminant_analysis import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

pipeRidge = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('cls', Ridge())
])

pipeRidge.fit(Xtrain,ytrain)
pred_pipeRidge = pipeRidge.predict(Xtest)


from sklearn.metrics import mean_squared_error

RMSE_pipeRidge = np.sqrt(mean_squared_error(ytest, pred_pipeRidge))

error_percent = 100 * (10**RMSE_pipeRidge - 1)
print(f'Average error for Ridge Pipeline is {error_percent:.2f}%')

Average error for Ridge Pipeline is 19.41%


In [95]:
list(features_test_corr.columns).index("House.Age"), list(features_test_corr.columns).index("Remod.Age"), list(features_test_corr.columns).index("Garage.Cars"), list(features_test_corr.columns).index("Garage.Age"), list(features_test_corr.columns).index("BsmtFin.SF.1")

(13, 12, 9, 11, 2)

In [107]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

class CombinedAttributesAdderCorrFeatures(BaseEstimator, TransformerMixin):
    # Column index.
    house_age, remodel, garage_cars, garage_age = "House.Age", "Remod.Age", "Garage.Cars", "Garage.Age"
    
    def __init__(self):  # No *args or **kwargs.
        pass
            
    def fit(self, X, y=None):
        return self  # Nothing else to do.

    def transform(self, X, y=None):

        df = X.copy()

        df['has_remodel'] = \
            df[CombinedAttributesAdderCorrFeatures.remodel] == df[CombinedAttributesAdderCorrFeatures.house_age]
        
        df['garage_by_house_age'] = \
            df[CombinedAttributesAdderCorrFeatures.garage_age] / df[CombinedAttributesAdderCorrFeatures.house_age]
        
        df = df.drop(columns = ["Remod.Age", "Garage.Cars"])

        return df.astype(float)

columns_std = ['X1st.Flr.SF','Total.Bsmt.SF','Lot.Frontage']


all_columns = list(features_test_corr.columns)

adder = Pipeline([
        ('imputer', SimpleImputer(strategy="mean")),
        ('attribs_adder', CombinedAttributesAdderCorrFeatures()),

    ])


full_pipeline = ColumnTransformer([
        ("scaler", StandardScaler(), columns_std),
        ("transform", adder, all_columns)
    ], remainder="passthrough")

Xtrain_transformed = full_pipeline.fit_transform(Xtrain)

Xtrain_transformed.shape

pipeRidge = Pipeline([
    # ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('cls', Ridge())
])

pipeRidge.fit(Xtrain_transformed,ytrain)

Xtest_transformed = full_pipeline.transform(Xtest)
pred_pipeRidge = pipeRidge.predict(Xtest_transformed)

from sklearn.metrics import mean_squared_error

RMSE_pipeRidge = np.sqrt(mean_squared_error(ytest, pred_pipeRidge))

error_percent = 100 * (10**RMSE_pipeRidge - 1)
print(f'Average error for Ridge Pipeline is {error_percent:.2f}%')

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices